In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import logging
import time
import json
from datetime import datetime
import psutil

# Skimage & Mahotas for feature extraction
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import regionprops
import mahotas as mt

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, make_scorer
)
from sklearn.svm import SVC
from sklearn.ensemble import (
    RandomForestClassifier, ExtraTreesClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# Add XGBoost as suggested in research plan
import xgboost as xgb

import joblib

import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

# CONFIGURATION

In [ ]:
# Global Configuration
ROOT_OUTPUT_DIR = "D:\iate_project\ml_output"
FEATURE_EXTRACTION_DIR = os.path.join(ROOT_OUTPUT_DIR, 'ml_features', 'ml_feature_extraction')
RESULTS_DIR = os.path.join(ROOT_OUTPUT_DIR, 'ml_classification_results')
MODELS_DIR = os.path.join(RESULTS_DIR, 'ml_models')
PLOTS_DIR = os.path.join(RESULTS_DIR, 'ml_plots')

In [ ]:
# Create necessary directories
for d in [ROOT_OUTPUT_DIR, FEATURE_EXTRACTION_DIR, RESULTS_DIR, MODELS_DIR, PLOTS_DIR]:
    os.makedirs(d, exist_ok=True)

In [ ]:
# Custom logging handler to filter progress bars from console output
class FilteredStreamHandler(logging.StreamHandler):
    def emit(self, record):
        if record.levelno >= logging.INFO:  # Only show INFO and above in console
            super().emit(record)

In [ ]:
# Set up logging - Drastically reduced console output
LOG_PATH = os.path.join(RESULTS_DIR, 'classification_and_feature_extraction.log')
file_handler = logging.FileHandler(LOG_PATH)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

console_handler = FilteredStreamHandler()
console_handler.setFormatter(logging.Formatter('>>> %(message)s'))

logger = logging.getLogger('coffee_bean_classification')
logger.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)
logger.propagate = False

In [ ]:
# Define classifiers with optimized hyperparameters based on research
CLASSIFIERS = {
    'SVM': {
        'model': SVC(
            C=150,
            kernel='rbf',
            probability=True,
            random_state=42,
            cache_size=1000
        ),
        'expects_proba': True,
    },
    'RandomForest': {
        'model': RandomForestClassifier(
            n_estimators=400,
            max_depth=20,
            min_samples_split=10,
            min_samples_leaf=4,
            max_features='log2',
            n_jobs=-1,
            random_state=42
        ),
        'expects_proba': True,
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(
            n_estimators=400,
            max_depth=20,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            objective='binary:logistic',
            random_state=42,
            n_jobs=-1
        ),
        'expects_proba': True,
    },
    'ExtraTrees': {
        'model': ExtraTreesClassifier(
            n_estimators=600,
            max_depth=50,
            min_samples_split=10,
            min_samples_leaf=4,
            max_features='log2',
            n_jobs=-1,
            random_state=42
        ),
        'expects_proba': True
    },
    'KNN': {
        'model': KNeighborsClassifier(
            n_neighbors=9,
            weights='distance',
            algorithm='auto',
            metric='euclidean',
            n_jobs=-1
        ),
        'expects_proba': True,
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(
            max_depth=50,
            min_samples_split=10,
            min_samples_leaf=4,
            random_state=42
        ),
        'expects_proba': True
    }
}

In [ ]:
# Define feature categories as mentioned in research plan
FEATURE_CATEGORIES = {
    'color': ['correlogram_'],  # Color features using correlogram
    'texture': ['glcm_'],       # Texture features using GLCM
    'shape': ['area', 'perimeter', 'eccentricity', 'extent', 'solidity']  # Shape features
}

# INITIALIZATION

In [ ]:
print("\n" + "="*70)
print("     COFFEE BEAN CLASSIFICATION WITH TRADITIONAL MACHINE LEARNING")
print("="*70)
logger.info("Starting coffee bean classification pipeline")

In [ ]:
# Record resource usage at the beginning
process = psutil.Process(os.getpid())
cpu_usage_before = psutil.cpu_percent(interval=None)
mem_info_before = process.memory_info()
memory_usage_mb_before = mem_info_before.rss / 1024 / 1024

start_time_script = time.time()

In [ ]:
# Feature Extraction Configuration
do_feature_extraction = True

In [ ]:
# Define path structure based on dataset description
original_dataset_path = "D:\iate_project\original_dataset"
output_dir = FEATURE_EXTRACTION_DIR
os.makedirs(output_dir, exist_ok=True)

# FEATURE EXTRACTION

In [ ]:
df = None

if do_feature_extraction:
    print("\n" + "-"*70)
    print("     STAGE 1: FEATURE EXTRACTION")
    print("-"*70)
    logger.info("Feature Extraction started")

    # Timers for each method
    method_times = {
        'color_correlogram': 0.0,
        'glcm': 0.0,
        'region_props': 0.0
    }

    # Lists to hold extracted data
    feats_list = []
    labels_ = []
    paths_ = []
    splits_ = []

    t_start_fe = time.time()

    # Create storage for class distribution info
    class_counts = {'train': {'Normal': 0, 'Defect': 0},
                    'test': {'Normal': 0, 'Defect': 0}}

    # Process image directories based on the specified structure
    classes = ['normal', 'defect']
    processing_types = ['dry', 'honey', 'wet']
    roast_levels = ['dark', 'light', 'medium']

    # Define train/test split ratio (70% train, 30% test)
    train_ratio = 0.7

    # Count total files for progress tracking
    total_files_count = 0
    for class_name in classes:
        for proc_type in processing_types:
            for roast in roast_levels:
                subfolder_path = os.path.join(original_dataset_path, class_name, proc_type, roast)
                if os.path.exists(subfolder_path):
                    image_files = [f for f in os.listdir(subfolder_path)
                                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    total_files_count += len(image_files)

    print(f"Total images to process: {total_files_count}")
    print("Starting feature extraction (this may take some time)...")

    # Track processing times and counts for occasional updates
    last_update_time = time.time()
    processed_since_update = 0
    total_processed = 0

    # Walk through the directory structure and extract features
    for class_name in classes:
        for proc_type in processing_types:
            for roast in roast_levels:
                # Construct the path to the current subfolder
                subfolder_path = os.path.join(original_dataset_path, class_name, proc_type, roast)

                if not os.path.exists(subfolder_path):
                    continue

                # Get list of image files in the current subfolder
                image_files = [f for f in os.listdir(subfolder_path)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                if not image_files:
                    continue

                # Shuffle files for random split
                np.random.seed(42)  # For reproducibility
                np.random.shuffle(image_files)

                # Split into train and test
                split_idx = int(len(image_files) * train_ratio)
                train_files = image_files[:split_idx]
                test_files = image_files[split_idx:]

                # Update class distribution counts
                class_counts['train'][class_name] += len(train_files)
                class_counts['test'][class_name] += len(test_files)

                # Process each split
                for split_name, files in [('train', train_files), ('test', test_files)]:
                    batch_start = time.time()
                    batch_size = len(files)

                    # Log batch info - for log file only
                    logger.info(f"Processing {batch_size} images from {class_name}/{proc_type}/{roast}/{split_name}")

                    # Show batch progress in console
                    print(f"Processing {batch_size} images from {class_name}/{proc_type}/{roast}/{split_name}")

                    for file_ in files:
                        # Construct full path to the image
                        img_path = os.path.join(subfolder_path, file_)

                        # Read and process the image
                        try:
                            # Read color image
                            color_img_ = cv2.imread(img_path, cv2.IMREAD_COLOR)
                            if color_img_ is None:
                                total_processed += 1
                                processed_since_update += 1
                                continue

                            # Convert BGR to RGB
                            color_img_ = cv2.cvtColor(color_img_, cv2.COLOR_BGR2RGB)

                            # Create grayscale image
                            gray_img_ = cv2.cvtColor(color_img_, cv2.COLOR_RGB2GRAY)

                            # Create threshold image (Otsu's method)
                            _, thresh_img_ = cv2.threshold(gray_img_, 0, 255,
                                                        cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                            # Extract features
                            # 1. Color correlogram - as referenced in research plan
                            t0_ = time.time()
                            correlogram = []
                            for d in [1, 2, 3]:  # Distances
                                for i in range(3):  # RGB channels
                                    channel = color_img_[:, :, i]
                                    haralick = mt.features.haralick(channel, distance=d)
                                    correlogram.extend(haralick.mean(axis=0).tolist())
                            cost_c = time.time() - t0_
                            method_times['color_correlogram'] += cost_c

                            # 2. GLCM features - as referenced in research plan
                            t1_ = time.time()
                            glcm_ = graycomatrix(
                                gray_img_,
                                distances=[1, 2, 3],
                                angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                                symmetric=True,
                                normed=True
                            )
                            glcm_features = []
                            props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
                            for prop in props:
                                val_ = graycoprops(glcm_, prop)
                                glcm_features.extend(val_.flatten().tolist())
                            cost_g = time.time() - t1_
                            method_times['glcm'] += cost_g

                            # 3. Region properties - shape features from research plan
                            t2_ = time.time()
                            regions = regionprops(thresh_img_.astype(np.uint8))
                            if not regions:
                                region_features = [0] * 5
                            else:
                                props_ = regions[0]
                                region_features = [
                                    props_.area,
                                    props_.perimeter,
                                    props_.eccentricity,
                                    props_.extent,
                                    props_.solidity
                                ]
                            cost_r = time.time() - t2_
                            method_times['region_props'] += cost_r

                            # Combine all features
                            flatten_vec = correlogram + glcm_features + region_features
                            feats_list.append(flatten_vec)
                            labels_.append(class_name)
                            paths_.append(os.path.join(class_name, proc_type, roast, file_))
                            splits_.append(split_name)

                        except Exception as e:
                            logger.error(f"Error processing {img_path}: {str(e)}")

                        # Update progress tracking
                        total_processed += 1
                        processed_since_update += 1

                        # Provide occasional progress updates
                        current_time = time.time()
                        if current_time - last_update_time > 60:  # Update every minute
                            percent_done = (total_processed / total_files_count) * 100
                            elapsed = current_time - t_start_fe
                            rate = processed_since_update / (current_time - last_update_time)
                            remaining = (total_files_count - total_processed) / rate if rate > 0 else 0

                            print(f"Progress: {total_processed}/{total_files_count} images ({percent_done:.1f}%), "
                                  f"Rate: {rate:.1f} img/sec, Est. remaining: {remaining/60:.1f} min")

                            last_update_time = current_time
                            processed_since_update = 0

                    # Batch completion summary
                    batch_time = time.time() - batch_start
                    print(f"Completed batch in {batch_time:.1f} seconds ({batch_size/batch_time:.1f} img/sec)")

    print("Feature extraction completed")

    # Check if we got any data
    if not feats_list:
        logger.error("No features extracted from any images.")
        print("ERROR: No features were extracted. Check the logs for details.")
        import sys
        sys.exit(-1)

    # Generate feature column names
    feat_names = (
        [f'correlogram_{i}' for i in range(13*3*3)] +  # 117
        [f'glcm_{j}' for j in range(5*3*4)] +         # 60
        ['area', 'perimeter', 'eccentricity', 'extent', 'solidity']  # 5
    )

    # Create DataFrame
    df = pd.DataFrame(feats_list, columns=feat_names)
    df['label'] = labels_
    df['image_path'] = paths_
    df['split'] = splits_

    # Split and scale
    print("Scaling features...")
    train_df = df[df['split'] == 'train']
    test_df = df[df['split'] == 'test']
    scaler = StandardScaler()
    train_mat = scaler.fit_transform(train_df[feat_names])
    test_mat = scaler.transform(test_df[feat_names])
    df.loc[df['split'] == 'train', feat_names] = train_mat
    df.loc[df['split'] == 'test', feat_names] = test_mat

    # Save scaler info
    sc_info = {
        'mean': scaler.mean_.tolist(),
        'scale': scaler.scale_.tolist(),
        'feature_names': feat_names
    }
    with open(os.path.join(output_dir, 'scaler_params.json'), 'w') as f_:
        json.dump(sc_info, f_, indent=4)

    # Summaries
    n_images_ = len(df)
    total_spent_ = time.time() - t_start_fe
    avg_times_ = {k: (method_times[k]/n_images_) for k in method_times}
    method_times['total_time'] = total_spent_
    method_times['average_times'] = avg_times_

    # Save CSV of extracted features
    output_csv_path = os.path.join(output_dir, 'coffee_bean_features.csv')
    df.to_csv(output_csv_path, index=False)

    # Build metadata
    class_dist = df.groupby(['split', 'label']).size()
    class_dist_dict = {
        f"{sp}_{lb}": cnt
        for (sp, lb), cnt in class_dist.items()
    }
    feature_dims = {
        'color_correlogram': 13*3*3,  # 117
        'glcm': 5*3*4,               # 60
        'region_props': 5
    }
    meta_ = {
        'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'processing_time': {
            'total_time': f"{method_times['total_time']:.2f} seconds",
            'method_breakdown': {
                k: f"{method_times[k]:.2f} seconds"
                for k in method_times
                if k not in ['total_time', 'average_times']
            },
            'average_times': {
                k: f"{avg_times_[k]:.4f} seconds/image"
                for k in avg_times_
                if k not in ['total_time', 'average_times']
            }
        },
        'dataset_statistics': {
            'total_samples': len(df),
            'class_distribution': class_dist_dict,
            'feature_dimensions': feature_dims,
            'total_dimensions': sum(feature_dims.values())
        },
        'parameters': {
            'color_correlogram_distances': [1, 2, 3],
            'glcm_distances': [1, 2, 3],
            'glcm_angles': [0, 45, 90, 135]
        }
    }

    meta_path_ = os.path.join(output_dir, 'extraction_metadata.json')
    with open(meta_path_, 'w') as f_:
        json.dump(meta_, f_, indent=4)

    # Print extraction summary
    print("\n" + "-"*70)
    print("                  FEATURE EXTRACTION SUMMARY")
    print("-"*70)
    print(f"Total samples: {len(df)} images")
    print(f"Feature dimensionality: {meta_['dataset_statistics']['total_dimensions']} features")
    print(f"  - Color features (correlogram): {feature_dims['color_correlogram']} dimensions")
    print(f"  - Texture features (GLCM): {feature_dims['glcm']} dimensions")
    print(f"  - Shape features: {feature_dims['region_props']} dimensions")
    print("\nClass distribution:")

    for s_ in ['train', 'test']:
        dist_s_ = df[df['split'] == s_]['label'].value_counts()
        print(f"  {s_:<5} set => Normal: {dist_s_.get('Normal', 0):<5}, Defect: {dist_s_.get('Defect', 0):<5}")

    print(f"\nTotal processing time: {method_times['total_time']:.1f} seconds ({n_images_/method_times['total_time']:.1f} images/sec)")

else:
    print("Skipping feature extraction. Loading features from CSV...")
    csv_path = os.path.join(FEATURE_EXTRACTION_DIR, "coffee_bean_features.csv")
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        logger.info(f"Loaded features from {csv_path}")
        print(f"Loaded {len(df)} samples from {csv_path}")
    else:
        logger.error(f"No feature CSV found at {csv_path}. Cannot proceed.")
        print(f"ERROR: No feature CSV found at {csv_path}. Cannot proceed.")
        import sys
        sys.exit(-1)

if df is None:
    logger.error("No DataFrame available for classification. Aborting.")
    print("ERROR: No data available for classification. Aborting.")
    import sys
    sys.exit(-1)

# CLASSIFICATION

In [ ]:
print("\n" + "-"*70)
print("     STAGE 2: CLASSIFICATION AND MODEL EVALUATION")
print("-"*70)
logger.info("Classification stage started")

In [ ]:
feat_cols = [c for c in df.columns if c not in ['label', 'image_path', 'split']]
train_df_ = df[df['split'] == 'train']
test_df_ = df[df['split'] == 'test']

X_train = train_df_[feat_cols].values
X_test = test_df_[feat_cols].values
label_map = {'Normal': 0, 'Defect': 1}
y_train = np.array([label_map[l] for l in train_df_['label']])
y_test = np.array([label_map[l] for l in test_df_['label']])

print(f"Dataset prepared for classification:")
print(f"  - Train set: {len(X_train)} samples")
print(f"  - Test set: {len(X_test)} samples")
print(f"  - Features: {len(feat_cols)} dimensions")

In [ ]:
# Define feature combinations to evaluate
feature_combos = [
    ['color'],
    ['texture'],
    ['shape'],
    ['color', 'texture'],
    ['color', 'shape'],
    ['texture', 'shape'],
    ['color', 'texture', 'shape']
]

# Setup for results collection
all_results = {}
best_model_script = {
    'score': -1,
    'combo': None,
    'clf_name': None,
    'model': None,
    'results': None,
    'features': None
}

print("\nEvaluating feature combinations and classifiers...")
print("This will test 7 feature combinations × 7 classifiers = 49 models")

In [ ]:
# Set up evaluation progress tracking
total_evals = len(feature_combos) * len(CLASSIFIERS)
start_eval_time = time.time()
completed_evals = 0

In [ ]:
# Loop through each feature combination
for combo_ in feature_combos:
    # Select features for this combination
    idx_sel_ = []
    dims_ = {}

    # Select feature indices for current combination
    for cat_ in combo_:
        cat_cols_ = []
        for prefix_ in FEATURE_CATEGORIES[cat_]:
            if prefix_.endswith('_'):
                found_ = [i for i, c in enumerate(feat_cols) if c.startswith(prefix_)]
            else:
                found_ = [i for i, c in enumerate(feat_cols) if c == prefix_]
            cat_cols_.extend(found_)
        dims_[cat_] = len(cat_cols_)
        idx_sel_.extend(cat_cols_)

    # Extract selected features
    X_train_sel_ = X_train[:, idx_sel_]
    X_test_sel_ = X_test[:, idx_sel_]
    sel_names_ = [feat_cols[i] for i in idx_sel_]

    combo_name_ = '+'.join(combo_)
    all_results[combo_name_] = {}

    # Compact feature info log
    feature_info = ", ".join([f"{c}: {dims_[c]}" for c in combo_])
    print(f"\nEvaluating feature combination: '{combo_name_}' ({feature_info})")

    # Evaluate each classifier with this feature combination
    for clf_name_, clf_info_ in CLASSIFIERS.items():
        eval_start = time.time()

        # Create a pipeline with just the classifier
        pipeline_ = Pipeline([
            ('classifier', clf_info_['model'])
        ])

        # Cross-validation - Changed from 5-fold to 10-fold
        cv_metrics_ = {}
        splitter_ = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scoring_ = {
            'accuracy': 'accuracy',
            'precision': make_scorer(precision_score, zero_division=0),
            'recall': 'recall',
            'f1': 'f1',
            'roc_auc': 'roc_auc'
        }
        cv_results_ = cross_validate(
            pipeline_, X_train_sel_, y_train,
            scoring=scoring_, cv=splitter_, n_jobs=-1, return_train_score=True
        )

        # Process CV results
        for metric, scores in [(k.replace('test_', ''), cv_results_[k])
                               for k in cv_results_ if k.startswith('test_')]:
            cv_metrics_[metric] = {
                'mean': float(scores.mean()),
                'std': float(scores.std()),
                'values': scores.tolist()
            }

        # Final fit on entire train set
        pipeline_.fit(X_train_sel_, y_train)

        # Predict test set
        y_pred_ = pipeline_.predict(X_test_sel_)
        y_proba_ = None
        if clf_info_['expects_proba']:
            y_proba_ = pipeline_.predict_proba(X_test_sel_)[:, 1]

        # Compute test metrics
        metrics_ = {
            'accuracy': accuracy_score(y_test, y_pred_),
            'precision': precision_score(y_test, y_pred_),
            'recall': recall_score(y_test, y_pred_),
            'f1': f1_score(y_test, y_pred_),
            'roc_auc': roc_auc_score(y_test, y_proba_) if y_proba_ is not None else None
        }

        # Get feature importances if available
        final_clf_ = pipeline_.named_steps['classifier']
        feat_imp_ = None
        if hasattr(final_clf_, 'feature_importances_'):
            feat_imp_ = final_clf_.feature_importances_
        elif hasattr(final_clf_, 'coef_'):
            feat_imp_ = np.abs(final_clf_.coef_[0])

        # Record total time
        total_t_ = time.time() - eval_start

        # Store results
        result_dict_ = {
            'model_name': clf_name_,
            'cross_validation': cv_metrics_,
            'test_performance': metrics_,
            'timing': {
                'total_time': total_t_
            },
            'feature_importance': feat_imp_.tolist() if feat_imp_ is not None else None
        }
        all_results[combo_name_][clf_name_] = result_dict_

        # Track the best model by F1 score
        cur_f1_ = metrics_['f1']
        if cur_f1_ > best_model_script['score']:
            best_model_script['score'] = cur_f1_
            best_model_script['combo'] = combo_
            best_model_script['clf_name'] = clf_name_
            best_model_script['model'] = pipeline_
            best_model_script['results'] = result_dict_
            best_model_script['features'] = sel_names_

        # Update progress
        completed_evals += 1
        elapsed = time.time() - start_eval_time
        rate = completed_evals / elapsed if elapsed > 0 else 0
        remaining = (total_evals - completed_evals) / rate if rate > 0 else 0

        # Print result with clear formatting, highlight best so far
        is_best = (best_model_script['clf_name'] == clf_name_ and
                   best_model_script['combo'] == combo_)

        best_marker = "★ " if is_best else "  "

        print(f"{best_marker}{clf_name_:15} | "
              f"F1: {metrics_['f1']:.4f} | "
              f"Acc: {metrics_['accuracy']:.4f} | "
              f"Prec: {metrics_['precision']:.4f} | "
              f"Rec: {metrics_['recall']:.4f} | "
              f"Time: {total_t_:.1f}s")

logger.info("All models evaluated successfully.")

In [ ]:
# Create a summary table
summary_table = []
for combo_key, model_dict in all_results.items():
    for model_k, model_res in model_dict.items():
        summary_table.append({
            'Feature Combo': combo_key,
            'Model': model_k,
            'Accuracy': f"{model_res['test_performance']['accuracy']:.4f}",
            'Precision': f"{model_res['test_performance']['precision']:.4f}",
            'Recall': f"{model_res['test_performance']['recall']:.4f}",
            'F1 Score': f"{model_res['test_performance']['f1']:.4f}",
            'ROC AUC': (
                f"{model_res['test_performance']['roc_auc']:.4f}"
                if model_res['test_performance']['roc_auc'] is not None
                else 'N/A'
            )
        })

summary_df = pd.DataFrame(summary_table)

In [ ]:
# Print highly visible results banner
print("\n" + "="*70)
print("                         RESULTS SUMMARY")
print("="*70)
print(f"\nBEST MODEL: {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
print(f"F1 SCORE: {best_model_script['score']:.4f}")
print(f"ACCURACY: {best_model_script['results']['test_performance']['accuracy']:.4f}")
print(f"PRECISION: {best_model_script['results']['test_performance']['precision']:.4f}")
print(f"RECALL: {best_model_script['results']['test_performance']['recall']:.4f}")

In [ ]:
# Show top 3 models
print("\nTOP 3 MODELS BY F1 SCORE:")
top_models = sorted(
    [(combo, model, results['test_performance']['f1'])
     for combo, models in all_results.items()
     for model, results in models.items()],
    key=lambda x: x[2], reverse=True
)[:3]

for i, (combo, model, f1) in enumerate(top_models, 1):
    result = all_results[combo][model]['test_performance']
    print(f"{i}. {model} with {combo}")
    print(f"   F1: {f1:.4f} | Acc: {result['accuracy']:.4f} | "
          f"Prec: {result['precision']:.4f} | Rec: {result['recall']:.4f}")

In [ ]:
# Save results as JSON
out_json_ = os.path.join(RESULTS_DIR, 'classification_results.json')
with open(out_json_, 'w') as f_:
    json.dump(all_results, f_, indent=4)

# Save best model
model_path_ = os.path.join(MODELS_DIR, 'best_model.joblib')
joblib.dump(best_model_script['model'], model_path_)

In [ ]:
# Save a detailed report
best_report_ = {
    'model_summary': {
        'name': best_model_script['clf_name'],
        'feature_combination': '+'.join(best_model_script['combo']),
        'total_features': len(best_model_script['features'])
    },
    'performance_metrics': best_model_script['results']['test_performance'],
    'timing_information': best_model_script['results']['timing'],
    'feature_importance': (
        dict(zip(
            best_model_script['features'],
            best_model_script['results']['feature_importance']
        )) if best_model_script['results']['feature_importance'] is not None else None
    )
}
report_path_ = os.path.join(RESULTS_DIR, 'detailed_report.json')
with open(report_path_, 'w') as f_:
    json.dump(best_report_, f_, indent=4)

print("\nClassification results and best model saved to disk.")

# VISUALIZATION

In [ ]:
print("\n" + "-"*70)
print("     STAGE 3: VISUALIZATION")
print("-"*70)

In [ ]:
# Generate best model plots
# Select features for best model
idx_sel_ = []
for cat_ in best_model_script['combo']:
    for prefix_ in FEATURE_CATEGORIES[cat_]:
        if prefix_.endswith('_'):
            found_ = [i for i, c in enumerate(feat_cols) if c.startswith(prefix_)]
        else:
            found_ = [i for i, c in enumerate(feat_cols) if c == prefix_]
        idx_sel_.extend(found_)
X_test_best_ = X_test[:, idx_sel_]

In [ ]:
# Get predictions
y_pred_best_ = best_model_script['model'].predict(X_test_best_)
y_proba_best_ = None
if hasattr(best_model_script['model'].named_steps['classifier'], 'predict_proba'):
    if best_model_script['clf_name'] in CLASSIFIERS and CLASSIFIERS[best_model_script['clf_name']]['expects_proba']:
        y_proba_best_ = best_model_script['model'].predict_proba(X_test_best_)[:,1]

In [ ]:
# Plot confusion matrix
print("Generating confusion matrix...")
cm_path_ = os.path.join(PLOTS_DIR, 'best_model_confusion_matrix.png')
cm_ = confusion_matrix(y_test, y_pred_best_)
classes_ = ['Normal', 'Defect']
disp_ = ConfusionMatrixDisplay(cm_, display_labels=classes_)
fig_, ax_ = plt.subplots(figsize=(8, 7))
disp_.plot(ax=ax_, cmap=plt.cm.Blues, colorbar=False)
plt.title(f"Confusion Matrix - {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
plt.tight_layout()
plt.savefig(cm_path_)
plt.close()

In [ ]:
# Print confusion matrix values
tn, fp, fn, tp = cm_.ravel()
print("\nConfusion Matrix:")
print(f"  True Negatives (Normal correctly predicted): {tn}")
print(f"  False Positives (Normal incorrectly as Defect): {fp}")
print(f"  False Negatives (Defect incorrectly as Normal): {fn}")
print(f"  True Positives (Defect correctly predicted): {tp}")

In [ ]:
# Plot ROC curve if probability estimates available
if y_proba_best_ is not None:
    print("Generating ROC curve...")
    roc_path_ = os.path.join(PLOTS_DIR, 'best_model_roc_curve.png')
    disp_ = RocCurveDisplay.from_predictions(y_test, y_proba_best_)
    fig_ = disp_.figure_
    plt.title(f"ROC Curve - {best_model_script['clf_name']} with {'+'.join(best_model_script['combo'])}")
    plt.tight_layout()
    plt.savefig(roc_path_)
    plt.close()

In [ ]:
# Plot feature importances if available
if best_model_script['results']['feature_importance'] is not None:
    print("Generating feature importance visualization...")
    fi_path_ = os.path.join(PLOTS_DIR, 'best_model_feature_importances.png')
    fi_abs_ = np.abs(np.array(best_model_script['results']['feature_importance']))
    idx_sorted_ = np.argsort(fi_abs_)[::-1]
    top_n = min(15, len(fi_abs_))
    fi_sorted_ = fi_abs_[idx_sorted_][:top_n]
    names_sorted_ = [best_model_script['features'][i] for i in idx_sorted_[:top_n]]

    fig_, ax_ = plt.subplots(figsize=(10, 8))
    y_pos_ = np.arange(len(fi_sorted_))
    ax_.barh(y_pos_, fi_sorted_[::-1], align='center', color='skyblue')
    ax_.set_yticks(y_pos_)
    ax_.set_yticklabels(names_sorted_[::-1])
    ax_.invert_yaxis()
    ax_.set_xlabel('Importance')
    ax_.set_title(f'Feature Importances - Top {top_n}')
    plt.tight_layout()
    plt.savefig(fi_path_)
    plt.close()

    # Print top features
    print("\nTop 5 Most Important Features:")
    for i in range(min(5, top_n)):
        print(f"  {i+1}. {names_sorted_[i]}: {fi_sorted_[i]:.4f}")

# FINAL SUMMARY

In [1]:
# Calculate final resource usage
cpu_usage_after = psutil.cpu_percent(interval=None)
mem_info_after = process.memory_info()
memory_usage_mb_after = mem_info_after.rss / 1024 / 1024

total_time_script = time.time() - start_time_script

print("\n" + "="*70)
print("                    PIPELINE COMPLETED")
print("="*70)
print(f"Total execution time: {total_time_script:.1f} seconds ({total_time_script/60:.1f} minutes)")
print(f"Memory usage: {memory_usage_mb_before:.1f}MB → {memory_usage_mb_after:.1f}MB")
print(f"All results saved to: {RESULTS_DIR}")
print("="*70)

>>> Starting coffee bean classification pipeline
>>> Feature Extraction started
>>> Processing 280 images from Normal/Dry/Dark/train



     COFFEE BEAN CLASSIFICATION WITH TRADITIONAL MACHINE LEARNING

----------------------------------------------------------------------
     STAGE 1: FEATURE EXTRACTION
----------------------------------------------------------------------
Total images to process: 5400
Starting feature extraction (this may take some time)...
Processing 280 images from Normal/Dry/Dark/train


>>> Processing 120 images from Normal/Dry/Dark/test


Completed batch in 50.2 seconds (5.6 img/sec)
Processing 120 images from Normal/Dry/Dark/test
Progress: 337/5400 images (6.2%), Rate: 5.6 img/sec, Est. remaining: 15.0 min


>>> Processing 280 images from Normal/Dry/Light/train


...

Completed batch in 24.6 seconds (5.7 img/sec)
Processing 60 images from Defect/Wet/Medium/test
Completed batch in 10.5 seconds (5.7 img/sec)
Feature extraction completed
Scaling features...


>>> Classification stage started



----------------------------------------------------------------------
                  FEATURE EXTRACTION SUMMARY
----------------------------------------------------------------------
Total samples: 5400 images
Feature dimensionality: 182 features
  - Color features (correlogram): 117 dimensions
  - Texture features (GLCM): 60 dimensions
  - Shape features: 5 dimensions

Class distribution:
  train set => Normal: 2520 , Defect: 1260 
  test  set => Normal: 1080 , Defect: 540  

Total processing time: 994.4 seconds (5.4 images/sec)

----------------------------------------------------------------------
     STAGE 2: CLASSIFICATION AND MODEL EVALUATION
----------------------------------------------------------------------
Dataset prepared for classification:
  - Train set: 3780 samples
  - Test set: 1620 samples
  - Features: 182 dimensions

Evaluating feature combinations and classifiers...
This will test 7 feature combinations × 7 classifiers = 49 models

Evaluating feature combina

>>> All models evaluated successfully.


  DecisionTree    | F1: 0.5955 | Acc: 0.7451 | Prec: 0.6320 | Rec: 0.5630 | Time: 1.5s

                         RESULTS SUMMARY

BEST MODEL: SVM with color+texture+shape
F1 SCORE: 0.8608
ACCURACY: 0.9123
PRECISION: 0.9146
RECALL: 0.8130

TOP 3 MODELS BY F1 SCORE:
1. SVM with color+texture+shape
   F1: 0.8608 | Acc: 0.9123 | Prec: 0.9146 | Rec: 0.8130
2. SVM with color+shape
   F1: 0.8420 | Acc: 0.9006 | Prec: 0.8956 | Rec: 0.7944
3. SVM with color+texture
   F1: 0.8374 | Acc: 0.8981 | Prec: 0.8947 | Rec: 0.7870

Classification results and best model saved to disk.

----------------------------------------------------------------------
     STAGE 3: VISUALIZATION
----------------------------------------------------------------------
Generating confusion matrix...

Confusion Matrix:
  True Negatives (Normal correctly predicted): 1039
  False Positives (Normal incorrectly as Defect): 41
  False Negatives (Defect incorrectly as Normal): 101
  True Positives (Defect correctly predicted): 4